In [50]:
from IPython.display import clear_output
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
!pip install -q tensorflow==2.4.1
clear_output()
import numpy as np
import pandas as pd
import tensorflow as tf 
from tensorflow import keras
import tensorflow_hub as hub

In [51]:
data_dir="../input/paddy-disease-classification/"
train =pd.read_csv(data_dir + "train.csv")
sub =pd.read_csv(data_dir + "sample_submission.csv")

In [52]:
train.head()

,image_id,label,variety,age
0,100330.jpg,bacterial_leaf_blight,ADT45,45
1,100365.jpg,bacterial_leaf_blight,ADT45,45
2,100382.jpg,bacterial_leaf_blight,ADT45,45
3,100632.jpg,bacterial_leaf_blight,ADT45,45
4,101918.jpg,bacterial_leaf_blight,ADT45,45


In [53]:
classes = np.unique(train["label"])
c2id = {c:i for i,c in enumerate(classes)}
id2c = {i:c for i,c in enumerate(classes)}
c2id,id2c

({'bacterial_leaf_blight': 0,
  'bacterial_leaf_streak': 1,
  'bacterial_panicle_blight': 2,
  'blast': 3,
  'brown_spot': 4,
  'dead_heart': 5,
  'downy_mildew': 6,
  'hispa': 7,
  'normal': 8,
  'tungro': 9},
 {0: 'bacterial_leaf_blight',
  1: 'bacterial_leaf_streak',
  2: 'bacterial_panicle_blight',
  3: 'blast',
  4: 'brown_spot',
  5: 'dead_heart',
  6: 'downy_mildew',
  7: 'hispa',
  8: 'normal',
  9: 'tungro'})

In [54]:
train["label_id"]=train["label"].map(c2id)
train

,image_id,label,variety,age,label_id
0,100330.jpg,bacterial_leaf_blight,ADT45,45,0
1,100365.jpg,bacterial_leaf_blight,ADT45,45,0
2,100382.jpg,bacterial_leaf_blight,ADT45,45,0
3,100632.jpg,bacterial_leaf_blight,ADT45,45,0
4,101918.jpg,bacterial_leaf_blight,ADT45,45,0
...,...,...,...,...,...
10402,107607.jpg,tungro,Zonal,55,9
10403,107811.jpg,tungro,Zonal,55,9
10404,108547.jpg,tungro,Zonal,55,9
10405,110245.jpg,tungro,Zonal,55,9


In [55]:
#Shuffle of file train.CSV
train = train.sample(frac=1.0)
train.head()

,image_id,label,variety,age,label_id
3667,103569.jpg,brown_spot,ADT45,72,4
8449,110116.jpg,normal,ADT45,60,8
577,102428.jpg,bacterial_leaf_streak,ADT45,77,1
2373,103669.jpg,blast,KarnatakaPonni,75,3
10001,104773.jpg,tungro,ADT45,70,9


In [56]:
train["image_id"]

3667     103569.jpg
8449     110116.jpg
577      102428.jpg
2373     103669.jpg
10001    104773.jpg
            ...    
7888     101562.jpg
7006     103074.jpg
7999     103098.jpg
4886     101032.jpg
6461     102031.jpg
Name: image_id, Length: 10407, dtype: object

In [57]:
#This is how we access to the working space (Input).
chemins = "train_images/"+train["label"]+"/"+train["image_id"].values
labels = train["label_id"].values
chemins,labels

(3667                train_images/brown_spot/103569.jpg
 8449                    train_images/normal/110116.jpg
 577      train_images/bacterial_leaf_streak/102428.jpg
 2373                     train_images/blast/103669.jpg
 10001                   train_images/tungro/104773.jpg
                              ...                      
 7888                    train_images/normal/101562.jpg
 7006                     train_images/hispa/103074.jpg
 7999                    train_images/normal/103098.jpg
 4886                train_images/dead_heart/101032.jpg
 6461                     train_images/hispa/102031.jpg
 Name: label, Length: 10407, dtype: object,
 array([4, 8, 1, ..., 8, 5, 7]))

In [58]:
train_ds = tf.data.Dataset.from_tensor_slices((chemins,labels))
train_ds

<TensorSliceDataset shapes: ((), ()), types: (tf.string, tf.int64)>

In [59]:
#extract et decode l'image @JPEG 
def extract_image(chemin,label):
    fichier = tf.io.read_file(data_dir +chemin)
    image=tf.image.decode_jpeg(fichier,channels=3)
    return image,label

In [60]:
train_ds =train_ds.map(extract_image)
train_ds

<MapDataset shapes: ((None, None, 3), ()), types: (tf.uint8, tf.int64)>

In [61]:
#normalisation de tf.unit8 a tf.float32
def normalisation(image,label):
    image = tf.cast(image,dtype=tf.float32)/255.0
    return image,label

In [62]:
train_ds =train_ds.map(normalisation)
train_ds

<MapDataset shapes: ((None, None, 3), ()), types: (tf.float32, tf.int64)>

In [63]:
def augment(image,label):
    image = tf.image.resize(image,size=(224,224))
    if tf.random.uniform(shape=[])<=0.5:
        image = tf.image.rot90(image,3)
    else:
        image = tf.image.flip_left_right(image)
    if tf.random.uniform(shape=[])<=0.7:
        image = tf.image.adjust_brightness(image,-0.2)
    return image,label

In [64]:
train_ds =train_ds.map(augment)
train_ds

<MapDataset shapes: ((224, 224, 3), ()), types: (tf.float32, tf.int64)>

In [65]:
train_ds =train_ds.batch(32)
train_ds

<BatchDataset shapes: ((None, 224, 224, 3), (None,)), types: (tf.float32, tf.int64)>

In [66]:
model=keras.Sequential()
model.add(hub.KerasLayer("https://tfhub.dev/google/cropnet/feature_vector/concat/1", trainable=True , arguments=dict(batch_norm_momentum=0.997)))
model.add(keras.layers.Dense(10,activation="softmax"))

In [67]:
opt = tf.keras.optimizers.SGD(learning_rate=0.01)

In [68]:
model.compile(optimizer=opt,loss="sparse_categorical_crossentropy",metrics=["accuracy"])

In [69]:
model.fit(train_ds,epochs=15)

Epoch 1/15
326/326 [==============================] - 67s 184ms/step - loss: 1.5891 - accuracy: 0.4667
Epoch 2/15
326/326 [==============================] - 60s 184ms/step - loss: 0.5889 - accuracy: 0.8110
Epoch 3/15
326/326 [==============================] - 59s 182ms/step - loss: 0.3690 - accuracy: 0.8875
Epoch 4/15
326/326 [==============================] - 60s 185ms/step - loss: 0.2360 - accuracy: 0.9292
Epoch 5/15
326/326 [==============================] - 60s 183ms/step - loss: 0.1616 - accuracy: 0.9548
Epoch 6/15
326/326 [==============================] - 61s 186ms/step - loss: 0.1191 - accuracy: 0.9660
Epoch 7/15
326/326 [==============================] - 60s 184ms/step - loss: 0.0847 - accuracy: 0.9772
Epoch 8/15
326/326 [==============================] - 60s 184ms/step - loss: 0.0611 - accuracy: 0.9858
Epoch 9/15
326/326 [==============================] - 60s 183ms/step - loss: 0.0448 - accuracy: 0.9912
Epoch 10/15
326/326 [==============================] - 60s 184ms/step - l

In [70]:
test_chemins = "test_images/"+sub["image_id"].values
test_labels = np.full(len(test_chemins),fill_value=-1)
test_chemins,test_labels

(array(['test_images/200001.jpg', 'test_images/200002.jpg',
        'test_images/200003.jpg', ..., 'test_images/203467.jpg',
        'test_images/203468.jpg', 'test_images/203469.jpg'], dtype=object),
 array([-1, -1, -1, ..., -1, -1, -1]))

In [71]:
test_ds =tf.data.Dataset.from_tensor_slices((test_chemins,test_labels))
test_ds

<TensorSliceDataset shapes: ((), ()), types: (tf.string, tf.int64)>

In [72]:
def augment2(image,label):
    image = tf.image.resize(image,size=(224,224))
    return image,label

In [73]:
test_ds = test_ds.map(extract_image).map(normalisation).map(augment2).batch(32)
test_ds

<BatchDataset shapes: ((None, 224, 224, 3), (None,)), types: (tf.float32, tf.int64)>

In [74]:
predictions = model.predict(test_ds)

In [75]:
predictions

array([[3.2789565e-05, 8.6439508e-07, 5.5678790e-08, ..., 9.9716216e-01,
        1.0727405e-05, 7.2789958e-06],
       [1.2636227e-06, 5.3168827e-07, 6.4858470e-09, ..., 9.6875028e-04,
        9.9902582e-01, 8.1454311e-07],
       [9.7294915e-06, 5.0708599e-04, 2.3169363e-05, ..., 3.5236561e-01,
        2.6443537e-02, 9.6999102e-06],
       ...,
       [9.6500639e-09, 6.2178116e-09, 3.8365571e-11, ..., 4.9744108e-07,
        9.9999940e-01, 1.4492154e-08],
       [2.7274407e-05, 9.9528354e-01, 2.3560176e-05, ..., 1.6336999e-03,
        6.4748670e-06, 7.9445941e-05],
       [2.1734694e-08, 2.3853872e-08, 3.1171273e-06, ..., 9.0239970e-05,
        1.0420945e-08, 2.2273248e-07]], dtype=float32)

In [76]:
sub["label_id"]=predictions.argmax(axis=1)
sub["label"]=sub["label_id"].map(id2c)
sub

,image_id,label,label_id
0,200001.jpg,hispa,7
1,200002.jpg,normal,8
2,200003.jpg,blast,3
3,200004.jpg,blast,3
4,200005.jpg,blast,3
...,...,...,...
3464,203465.jpg,dead_heart,5
3465,203466.jpg,hispa,7
3466,203467.jpg,normal,8
3467,203468.jpg,bacterial_leaf_streak,1


In [77]:
test_sub=sub[["image_id","label"]]
test_sub

,image_id,label
0,200001.jpg,hispa
1,200002.jpg,normal
2,200003.jpg,blast
3,200004.jpg,blast
4,200005.jpg,blast
...,...,...
3464,203465.jpg,dead_heart
3465,203466.jpg,hispa
3466,203467.jpg,normal
3467,203468.jpg,bacterial_leaf_streak


In [79]:
test_sub.to_csv("sub42.csv",index=False)